Camacho Castelan Jose Manuel
S22002245

In [178]:
import igraph
from igraph import Graph, EdgeSeq
import plotly.graph_objects as go

In [179]:
class Nodo():
    def __init__(self, estado, papa = None, costo = -1):
        global expandidos
        self.estado = estado
        self.hijos = []
        self.papa = papa
        self.heuristica = None
        self.costo = costo +1
        if self not in expandidos:
            expandidos.append(self)
        

    def genera_hijos(self, meta = None, metodo = None):
        global expandidos
        pos = self.estado.index("_")
        #respetar restricciones
        n = 3
        #posiciones
        # 0 1 2
        # 3 4 5
        # 6 7 8
        
        #mover arriba
        if (pos >= n):
            new_estado = self.estado[:]         # copiar
            new_estado[pos] = new_estado[pos-n] # copiar valor de esa posicion
            new_estado[pos-n] = "_"             # asignar el hueco en la matriz
            
            new_nodo = Nodo(new_estado, self, self.costo)
            self.hijos.append(new_nodo)
            
            if meta:
                if metodo == "greedy": 
                    new_nodo.heuristica2(meta)
                else:
                    new_nodo.f_n(meta)  # fn = g(n) (costo) + h(n) (heuristica)

        #mover abajo
        if (pos < n*2):
            new_estado = self.estado[:]         # copiar
            new_estado[pos] = new_estado[pos+n] # copiar valor de esa posicion
            new_estado[pos+n] = "_"             # asignar el hueco en la matriz

            new_nodo = Nodo(new_estado, self, self.costo)
            self.hijos.append(new_nodo)
            
            if meta:
                if metodo == "greedy":
                    new_nodo.heuristica2(meta)
                else:
                    new_nodo.f_n(meta)  # fn = g(n) (costo) + h(n)

        #mover izquierda
        if (pos%n != 0):
            new_estado = self.estado[:]         # copiar
            new_estado[pos] = new_estado[pos-1] # copiar valor de esa posicion
            new_estado[pos-1] = "_"             # asignar el hueco en la matriz

            new_nodo = Nodo(new_estado, self, self.costo)
            self.hijos.append(new_nodo)
            
            if meta:
                if metodo == "greedy":
                    new_nodo.heuristica2(meta)
                else:
                    new_nodo.f_n(meta)  # fn = g(n) (costo) + h(n)

        #mover derecha
        if ((pos+1)%n != 0):
            new_estado = self.estado[:]         # copiar
            new_estado[pos] = new_estado[pos+1] # copiar valor de esa posicion
            new_estado[pos+1] = "_"             # asignar el hueco en la matriz
 
            new_nodo = Nodo(new_estado, self, self.costo)
            self.hijos.append(new_nodo)
            
            if meta:
                
                if metodo == "greedy":
                    new_nodo.heuristica2(meta)
                else:
                    new_nodo.f_n(meta)  # fn = g(n) (costo) + h(n)

    def soy_visitado(self, visitados):
        #return any(self.estado == arr for arr in visitados)
        return self.estado in visitados

    def bpp(self, meta, visitados=None): # busqueda primero en profundidad (DFS Depth First Search)

        if self.estado == meta: # si soy
            visited = len(visitados)
            return [self]
        # soy gemelo malvado de uno visitado? checar si nodo ya se visito
        # return None
        # seguir buscando

        if visitados is None:
            visitados = []
        if self.soy_visitado(visitados): # ya lo visitamos
            return None

        res = None
        self.genera_hijos()
        visitados.append(self.estado)

        for h in self.hijos: # hijos de self
            res = h.bpp(meta, visitados) # los nietos 
            if not (res == None): # si hay resutado
                res.append(self)
                return res

    def bpa(self, meta, visitados = None, por_visitar = []): # busqueda primero por anchura
        global visited
        # --- hacerlo funcion
        if self.estado == meta: # soy la meta?
            return [self]
        
        if visitados is None: # si visitados vacio, se inicializa
            visitados = []
        if self.soy_visitado(visitados): # si soy visitado, true, y retorna none
            return None
        
        self.genera_hijos()
        visitados.append(self.estado)
        # --- hasta aqui

        por_visitar += self.hijos

        while (por_visitar != []):
            h = por_visitar.pop(0)
            if h.estado == meta: # soy yo?
                visited = len(por_visitar) + len(visitados)
                camino = [h]
                papa = h.papa
                while (papa != None):
                    camino.append(papa)
                    papa=papa.papa
                camino.reverse()
                return camino # checar aqui
        
            if h.soy_visitado(visitados): # ya lo visitamos?
                continue
            
            h.genera_hijos()
            visitados.append(h.estado)

            por_visitar += h.hijos

    def heuristica1(self, meta):
        contador = 0
        for e1, e2 in zip(self.estado, meta):
            if not e1 == e2:
                contador = contador + 1
        self.heuristica = contador
        return contador

    def heuristica2(self, meta):
        sumador = 0
        for el in self.estado:
            sumador += abs(self.estado.index(el) - meta.index(el))
        self.heuristica = sumador
        return sumador
            
    def f_n(self, meta):
        self.heuristica = self.costo + self.heuristica2(meta)
        return self.heuristica

    def greedy(self, meta, visitados = [], metodo = "greedy"):
        visitados = []
        #por_visitar = []
        por_visitar = [self]

        while(por_visitar!= []):
            h = por_visitar.pop(0)
            if h.estado == meta: # soy yo?
                camino = [h]
                papa = h.papa
                while (papa != None):
                    camino.append(papa)
                    papa=papa.papa
                camino.reverse()
                return camino # checar aqui
        
            if h.soy_visitado(visitados): # ya lo visitamos?
                continue
            
            h.genera_hijos(meta)
            visitados.append(h.estado)

            por_visitar += h.hijos
            por_visitar.sort()
            #ordenar los por_visitar

    def a_star(self, meta, visitados = []):
        visitados = []
        por_visitar = [self]

        while(por_visitar!= []):
            h = por_visitar.pop(0)
            if h.estado == meta: # soy yo?
                camino = [h]
                papa = h.papa
                while (papa != None):
                    camino.append(papa)
                    papa=papa.papa
                camino.reverse()
                return camino # checar aqui
        
            if h.soy_visitado(visitados): # ya lo visitamos?
                continue
            
            h.genera_hijos(meta)
            visitados.append(h.estado)

            por_visitar += h.hijos
            por_visitar.sort()
            #ordenar los por_visitar

    def size(self):
        visitados = []
        por_visitar = [self]

        while por_visitar != []:
            nodo = por_visitar.pop(0)
            if nodo in visitados:
                continue
            por_visitar += nodo.hijos
            visitados.append(nodo)
            
        return len(visitados)

    def __lt__(self, n2):
        if n2 == None:
            return False
        return self.heuristica < n2.heuristica

    def __eq__(self, n2):
        if n2 == None:
            return False
        return self.estado == n2.estado

    def __repr__(self):
        return("\n" + str(self.costo) + " --------\n" + str(self.estado[:3]) + "\n" + str(self.estado[3:6]) +"\n" + str(self.estado[6:]))


In [180]:
def get_nodos(raiz):
    nodos = []
    por_visitar = [raiz]
    visitados = []

    while por_visitar != []:
        nodo = por_visitar.pop(0)
        if (nodo in visitados) or (nodo in nodos):
            continue
        nodos.append(nodo)

        por_visitar += nodo.hijos
        visitados.append(nodo)

    return nodos

In [181]:
def get_etiquetas(nodos):
    labels = []
    for nodo in nodos:
        labels.append(
            str(nodo.estado[:3]) + "<br>" +
            str(nodo.estado[3:6]) + "<br>" +
            str(nodo.estado[6:])
        )
    return labels

In [182]:
def crear_grafo(nodos, etiquetas):
    
    g = Graph(directed = True)

    for nodo in nodos:
        g.add_vertex(etiquetas[nodos.index(nodo)])
    padre_hijo = []
    for nodo in nodos:
        
        padre_hijo.append([nodo.papa, nodo])
    for r in padre_hijo:
        if r[0]:  # tiene papa
            g.add_edge(etiquetas[nodos.index(r[0])], etiquetas[nodos.index(r[1])])

    edges = [e.tuple for e in g.es]
    
    return g, edges
    

In [183]:
def dibujar_grafo(grafo, labels):
    # Encontrar raíz: nodo sin padres (grado de entrada 0)
    raices = [v.index for v in grafo.vs if grafo.degree(v, mode="in") == 0]
    if len(raices) != 1:
        return f"SHay {len(raices)}, imposible ejecutar"
    root_idx = raices[0]

    layout = grafo.layout_reingold_tilford(mode="out", root=[root_idx]) # out, que no se graficque hacia los lados, sino hacia abajo

    Xn = [pos[0] for pos in layout]
    Yn = [-pos[1] for pos in layout]  # y negativa: hacia abajo

    Xe, Ye = [], []
    for edge in grafo.get_edgelist():
        Xe += [Xn[edge[0]], Xn[edge[1]], None]
        Ye += [Yn[edge[0]], Yn[edge[1]], None]

    trace_edges = go.Scatter(
        x=Xe, y=Ye,
        mode='lines',
        line=dict(width=1, color='gray'),
        hoverinfo='none'
    )

    trace_nodes = go.Scatter(
        x=Xn, y=Yn,
        mode='markers',
        marker=dict(symbol="circle-dot", size=30, color="lightblue", line=dict(width=2)),
        text=labels,
        textposition="middle center",
        hoverinfo="text"
    )

    fig = go.Figure(data=[trace_edges, trace_nodes],
                    layout=go.Layout(title="Arbol 8 puzzle",
                                     showlegend=True,
                                     margin=dict(b=20, l=5, r=5, t=40),
                                     xaxis=dict(showgrid=False, zeroline=False),
                                     yaxis=dict(showgrid=False, zeroline=False)))
    fig.show()


In [184]:
# BPA
meta = [1, 2, 3,
        4, 5, 6,
        7, 8, "_"]

estado = [4,1, 3,
          7,"_", 2,
          8, 6, 5]

In [185]:
expandidos = []

raiz = Nodo(estado)

camino = raiz.a_star(meta)
print("Movimientos:", len(camino) - 1)
print("Tamanio: " + str(len(expandidos)))

Movimientos: 10
Tamanio: 50


In [186]:
#nodos = get_nodos(expandidos)
etiquetas = get_etiquetas(expandidos)

In [187]:
grafo, edges = crear_grafo(expandidos, etiquetas)

Grafo general

In [188]:
dibujar_grafo(grafo, etiquetas)

In [189]:
etiquetas_camino = get_etiquetas(camino)

In [190]:
gC, edges = crear_grafo(camino, etiquetas_camino)

**Grafo de camino**

In [191]:
dibujar_grafo(gC, etiquetas_camino)